In [1]:
from __future__ import annotations

import json
import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RefMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
async def dummy_loader(name: str) -> dict:
    await sleep(0.2)
    return {
        'name': name,
        'age': 20
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RefMap(
            ref=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RefMap(
                    ref=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RefMap(
                            ref=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RefMap(
                    ref=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    rel=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [7]:
Task.assignees.ctx


Table(
    _db=DataBase(
        _db=...,
        _ctx=None,
        _name=None,
        _typehint=None,
        _typevar_map={},
        _filters=[],
        _tuple_selection=None,
        backend=<py_research.db.databases.Symbolic object at 0x0000027A52ED22D0>,
        url=None,
        types=None,
        schema=None,
        write_to_overlay=None,
        overlay_type='name_prefix',
        validate_on_init=False,
        remove_cross_fks=False,
        _def_types={},
        _subs={},
        _metadata=MetaData(),
        _valid_caches={},
        _instance_map={}
    ),
    _ctx=None,
    _name=None,
    _typehint=py_research.db.databases.Table[test_db_schema.Task, None, py_research.db.databases.CRUD, py_research.db.databases.BaseIdx, None, None],
    _typevar_map={},
    _filters=[],
    _tuple_selection=None,
    index_by=None,
    default=False
)

In [8]:
with cProfile.Profile() as pr:
  db = DataBase()
  rec = await data_source.load([data], db=db)
  
  stats = pstats.Stats(pr)

In [12]:
[s for s in db[Search][Search.result_count]]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 [s for s in db[Search][Search.result_count]]                                                 │
│   2                                                                                              │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1170 in __iter__                │
│                                                                                                  │
│   1167 │   def __iter__(  # noqa: D105                                                           │
│   1168 │   │   self: Data[ValT2, Any, Any, Any, Any, DynBackendID],                              │
│   1169 │   ) -> Iterator[ValT2]:                                                                 │
│ ❱ 1170 │   │   return iter(self.values())                                                        │
│   1171 │                                                                                         │
│   1172 │   @overload                                                                             │
│   1173 │   def __imatmul__(                                                                      │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:688 in values                   │
│                                                                                                  │
│    685 │   │   self: Data[ValT2, Any, Any, Any, Any, DynBackendID],                              │
│    686 │   ) -> Sequence[ValT2]:                                                                 │
│    687 │   │   """Iterable over this dataset's values."""                                        │
│ ❱  688 │   │   dfs = self.to_df()                                                                │
│    689 │   │   if isinstance(dfs, pl.DataFrame):                                                 │
│    690 │   │   │   dfs = (dfs,)                                                                  │
│    691                                                                                           │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:802 in to_df                    │
│                                                                                                  │
│    799 │   │   index_only: bool = False,                                                         │
│    800 │   ) -> DfT:                                                                             │
│    801 │   │   """Load dataset as dataframe."""                                                  │
│ ❱  802 │   │   select = type(self).select(self, index_only=index_only)                           │
│    803 │   │                                                                                     │
│    804 │   │   merged_df = None                                                                  │
│    805 │   │   if kind is pd.DataFrame:                                                          │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\caching.py:371 in __call__                      │
│                                                                                                  │
│   368 │   │   │   assert isinstance(args[0], Callable)                                           │
│   369 │   │   │   return copy_and_override(cached_prop[Any,

In [ ]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         104641 function calls (98827 primitive calls) in 0.316 seconds

   Ordered by: cumulative time
   List reduced from 2074 to 126 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.068    0.034 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:4329(_update_dict)
        1    0.000    0.000    0.050    0.050 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:3497(engine)
        2    0.000    0.000    0.037    0.018 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:809(to_df)
    40/25    0.000    0.000    0.030    0.001 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1665(__get__)
       30    0.000    0.000    0.013    0.000 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2323(_has_type)
        3    0.000    0.000    0.012    0.004 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:688(_load_reco